Lambda School Data Science

*Unit 2, Sprint 1, Module 3*

---

# Ridge Regression

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

- [ o ] Use a subset of the data where `BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'` and the sale price was more than 100 thousand and less than 2 million.
- [ o ] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [ o ] Do one-hot encoding of categorical features.
- [ o ] Do feature selection with `SelectKBest`.
- [  ] Fit a ridge regression model with multiple features. Use the `normalize=True` parameter (or do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html) beforehand — use the scaler's `fit_transform` method with the train set, and the scaler's `transform` method with the test set)
- [ o ] Get mean absolute error for the test set.
- [ o ] As always, commit your notebook to your fork of the GitHub repo.

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.




In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import pandas as pd
import pandas_profiling

# Read New York City property sales data
df = pd.read_csv(DATA_PATH+'condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)

In [0]:
# BOROUGH is a numeric column, but arguably should be a categorical feature,
# so convert it from a number to a string
df['BOROUGH'] = df['BOROUGH'].astype(str)

In [0]:
# Reduce cardinality for NEIGHBORHOOD feature

# Get a list of the top 10 neighborhoods
top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10, 
# replace the neighborhood with 'OTHER'
df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'

In [100]:
# visualizing the code to get a handle of the conlumns and categorical variables. 
df.sample(2)

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
11377,2,OTHER,29 COMMERCIAL GARAGES,4,2573,39,NaN,G7,727 EAST 141 STREET,NaN,10454.0,0.0,0.0,0.0,"2,000",0.0,0.0,4,G7,420000,02/27/2019
20540,2,OTHER,09 COOPS - WALKUP APARTMENTS,2,2711,21,NaN,C6,"941 TIFFANY STREET, 2 FL",NaN,10459.0,0.0,0.0,0.0,0,0.0,1989.0,2,C6,0,04/12/2019


In [101]:
# confirm a starting shape for th DF before filtering. 
df.shape

(23040, 21)

### Further filtering of the dataset

Use a subset of the data where BUILDING_CLASS_CATEGORY == '01 ONE FAMILY DWELLINGS' and the sale price was more than 100 thousand and less than 2 million.

In [36]:
# checking the various datatypes. 
df.dtypes

BOROUGH                            object
NEIGHBORHOOD                       object
BUILDING_CLASS_CATEGORY            object
TAX_CLASS_AT_PRESENT               object
BLOCK                               int64
LOT                                 int64
EASE-MENT                         float64
BUILDING_CLASS_AT_PRESENT          object
ADDRESS                            object
APARTMENT_NUMBER                   object
ZIP_CODE                          float64
RESIDENTIAL_UNITS                 float64
COMMERCIAL_UNITS                  float64
TOTAL_UNITS                       float64
LAND_SQUARE_FEET                   object
GROSS_SQUARE_FEET                 float64
YEAR_BUILT                        float64
TAX_CLASS_AT_TIME_OF_SALE           int64
BUILDING_CLASS_AT_TIME_OF_SALE     object
SALE_PRICE                          int64
SALE_DATE                          object
dtype: object

In [0]:
# converts the object dtype to a string
df['BUILDING_CLASS_CATEGORY'] = df['BUILDING_CLASS_CATEGORY'].astype(str)

In [0]:
# convert a int64 to datetime so it can be filtered below. 
df['SALE_DATE'] = pd.to_datetime(df['SALE_DATE'], infer_datetime_format=True)

In [0]:
# filtering the dataset prior to doinga train/test split. 
df = df[(df['BUILDING_CLASS_CATEGORY'] == '01 ONE FAMILY DWELLINGS') & 
        (df['SALE_PRICE'] > 100000) & 
        (df['SALE_PRICE'] < 2000000)] 

- [ ] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.

In [0]:
# filtering the dataset by when it was created 
df = df.set_index(df['SALE_DATE'])
df = df.sort_index()

In [41]:
# setting it to January — March 2019 to train. Use data from April 2019 to test.
train_2019 = df['2019-01-01':'2019-03-30']
test_2019 = df['2019-04-01':'2019-04-30']
train_2019.shape, test_2019.shape

((2507, 21), (644, 21))

- [ ] Do one-hot encoding of categorical features.

In [0]:
# assigining the sale price as a target. 
target = 'SALE_PRICE'
y_train = train_2019[target]
y_test = test_2019[target]

In [44]:
# trying to figure out the categorical features that are relevant. 
train_2019['NEIGHBORHOOD'].unique()

array(['OTHER', 'FLUSHING-NORTH', 'BEDFORD STUYVESANT', 'FOREST HILLS',
       'BOROUGH PARK', 'ASTORIA'], dtype=object)

In [45]:
# trying to figure out the categorical features that are relevant. 
train_2019['BUILDING_CLASS_AT_TIME_OF_SALE'].unique()


array(['A9', 'A1', 'A5', 'A0', 'A2', 'A3', 'S1', 'A4', 'A6', 'A8', 'S0'],
      dtype=object)

In [0]:
# Performing a one-hot encoding. 
import category_encoders as ce 
features = ['NEIGHBORHOOD',  'BUILDING_CLASS_AT_TIME_OF_SALE']

X_train = train_2019[features]
X_test = test_2019[features]
encoder = ce.one_hot.OneHotEncoder(use_cat_names=True)
X_train_enc = encoder.fit_transform(X_train)
X_test_enc = encoder.transform(X_test) # not fit_transform

In [50]:
# visualizing the one-hot encoded df. 
X_train_enc

,NEIGHBORHOOD_OTHER,NEIGHBORHOOD_FLUSHING-NORTH,NEIGHBORHOOD_BEDFORD STUYVESANT,NEIGHBORHOOD_FOREST HILLS,NEIGHBORHOOD_BOROUGH PARK,NEIGHBORHOOD_ASTORIA,BUILDING_CLASS_AT_TIME_OF_SALE_A9,BUILDING_CLASS_AT_TIME_OF_SALE_A1,BUILDING_CLASS_AT_TIME_OF_SALE_A5,BUILDING_CLASS_AT_TIME_OF_SALE_A0,BUILDING_CLASS_AT_TIME_OF_SALE_A2,BUILDING_CLASS_AT_TIME_OF_SALE_A3,BUILDING_CLASS_AT_TIME_OF_SALE_S1,BUILDING_CLASS_AT_TIME_OF_SALE_A4,BUILDING_CLASS_AT_TIME_OF_SALE_A6,BUILDING_CLASS_AT_TIME_OF_SALE_A8,BUILDING_CLASS_AT_TIME_OF_SALE_S0
SALE_DATE,,,,,,,,,,,,,,,,,
2019-01-01,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2019-01-01,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2019-01-02,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2019-01-02,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2019-01-02,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-03-29,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2019-03-29,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2019-03-29,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


- [ ] Do feature selection with `SelectKBest`.


In [69]:
# visualizing the features that are relevant --> better code thanks TL's. 
df.describe(exclude='number').T.sort_values(by='unique')

,count,unique,top,freq,first,last
BUILDING_CLASS_CATEGORY,3151,1,01 ONE FAMILY DWELLINGS,3151,NaT,NaT
APARTMENT_NUMBER,1,1,RP.,1,NaT,NaT
TAX_CLASS_AT_PRESENT,3151,2,1,3111,NaT,NaT
BOROUGH,3151,5,4,1580,NaT,NaT
NEIGHBORHOOD,3151,6,OTHER,2990,NaT,NaT
BUILDING_CLASS_AT_TIME_OF_SALE,3151,11,A1,1186,NaT,NaT
BUILDING_CLASS_AT_PRESENT,3151,13,A1,1185,NaT,NaT
SALE_DATE,3151,91,2019-01-31 00:00:00,78,2019-01-01,2019-04-30
LAND_SQUARE_FEET,3151,1035,"4,000",289,NaT,NaT
ADDRESS,3151,3135,130-52 LEFFERTS BOULEVARD,2,NaT,NaT


In [0]:
# scales the dataset before linear regression processing. 
from sklearn.preprocessing import StandardScaler 
scaler = StandardScaler()
transform_train = scaler.fit_transform(X_train_enc)
transform_test = scaler.transform(X_test_enc)

In [72]:
# don't know why this is here. 
features = transform_train
n = len(features)
n

2507

In [0]:
# importing SelectKBest and f_regression. 
from sklearn.feature_selection import SelectKBest, f_regression

# doing the Select K-best
selector = SelectKBest(score_func=f_regression, k=15)
# .fit_transform on the train set
# .transform on test set
X_train_kbest = selector.fit_transform(transform_train, y_train)
X_test_kbest = selector.transform(transform_test)

In [75]:
# visualizing the shape of the train/test kbest
X_train_kbest.shape, X_test_kbest.shape


((2507, 15), (644, 15))

In [80]:
# Importing LinearRegression and mean_absolute_error. 
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

# finding the MAE of the linear-regression model. 
mae_list = []
for k in range(1, transform_train.shape[1] + 1):
  print(f'Using top {k} features')
  selector = SelectKBest(score_func=f_regression, k=k)
  X_train_kbest = selector.fit_transform(transform_train, y_train)
  X_test_kbest = selector.transform(transform_test)
  model = LinearRegression()
  model.fit(X_train_kbest, y_train)
  y_pred = model.predict(X_test_kbest)
  mae = mean_absolute_error(y_pred, y_test)
  print(f'MAE: {mae}')
  mae_list.append(mae)

Using top 1 features
MAE: 206569.95058579274
Using top 2 features
MAE: 198357.8951063684
Using top 3 features
MAE: 197864.48016501256
Using top 4 features
MAE: 197589.67068759486
Using top 5 features
MAE: 197676.14085468338
Using top 6 features
MAE: 197764.30644485657
Using top 7 features
MAE: 197363.2457972034
Using top 8 features
MAE: 196608.94816151675
Using top 9 features
MAE: 196560.17499811013
Using top 10 features
MAE: 193730.49551344058
Using top 11 features
MAE: 194347.94962370978
Using top 12 features
MAE: 195052.82151347256
Using top 13 features
MAE: 194870.00698979764
Using top 14 features
MAE: 194533.41303332802
Using top 15 features
MAE: 194985.9660186389
Using top 16 features
MAE: 195268.9108351187
Using top 17 features
MAE: 194812.40431503215


In [88]:
# getting the shape of the transformed dataset. 
transform_train.shape

(2507, 17)

In [93]:
# realize transform train is a numpy array. 
transform_train.sample()

AttributeError: ignored

In [90]:
# this plot has elected to permaghost. 
import seaborn as sns
sns.scatterplot(range(1, 18), mae_list)

In [95]:
# stacking code to essentially combine what we have done before but for ridge-regression. 
from sklearn.feature_selection import f_regression, SelectKBest
from sklearn.linear_model import RidgeCV
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler


# this wonderous bit of code spits out the mean-absolute-error of the ridgeCV model. 
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_enc)
X_test_scaled = scaler.transform(X_test_enc)
for k in range(1, len(X_train_enc.columns)+1):
    print(f'{k} features')
    selector = SelectKBest(score_func=f_regression, k=k)
    X_train_selected = selector.fit_transform(X_train_scaled, y_train)
    X_test_selected = selector.transform(X_test_scaled)
    model = RidgeCV()
    model.fit(X_train_selected, y_train)
    y_pred = model.predict(X_test_selected)
    mae = mean_absolute_error(y_test, y_pred)
    print(f'Test MAE: ${mae:,.0f} \n')

1 features
Test MAE: $206,583 

2 features
Test MAE: $198,384 

3 features
Test MAE: $197,886 

4 features
Test MAE: $197,646 

5 features
Test MAE: $197,726 

6 features
Test MAE: $197,810 

7 features
Test MAE: $197,410 

8 features
Test MAE: $196,626 

9 features
Test MAE: $196,577 

10 features
Test MAE: $193,753 

11 features
Test MAE: $194,353 

12 features
Test MAE: $195,009 

13 features
Test MAE: $194,831 

14 features
Test MAE: $194,498 

15 features
Test MAE: $194,935 

16 features
Test MAE: $194,940 

17 features
Test MAE: $195,005 



## Stretch Goals

Don't worry, you aren't expected to do all these stretch goals! These are just ideas to consider and choose from.

- [ ] Add your own stretch goal(s) !
- [ ] Instead of `Ridge`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [ ] Instead of `Ridge`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [0]:
#TODO